<a href="https://colab.research.google.com/github/stormyCobra1199/olist/blob/main/Olist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**bOwLISTIC 5**

Prep ipynb for OLIST data ingestion workflow.
To be used in conjunction with olistDAG.py DAG.


SCHEMA PLAN (9 Tables in total)

sellers:
*   seller_id str 
*   seller_zip_code_prefix int
*   seller_city str
*   seller_state str

customers:
*   customer_id str
*   customer_unique_id str
*   customer_zip_code_prefix int
*   customer_city str
*   customer_state str

category_name:
*   product_category_name str
*   product_category_name_english str

products:
*   product_id str
*   product_category_name str
*   product_name_lenght int
*   product_description_lenght int
*   product_photos_qty int
*   product_weight_g int
*   product_length_cm int
*   product_height_cm int
*   product_width_cm int

orders:
*   order_id str
*   customer_id str
*   order_status str
*   order_purchase_timestamp str
*   order_approved_at str
*   order_delivered_carrier_date str
*   order_delivered_customer_date str
*   order_estimated_delivery_date str

order_items:
*   order_id str
*   order_item_id int
*   product_id str
*   seller_id str
*   shipping_limit_date str
*   price float
*   freight_value float

order_payments:
*   order_id str
*   payment_sequential int
*   payment_type str
*   payment_installments int
*   payment_value float

order_reviews:
*   review_id str
*   order_id str
*   review_score int
*   review_comment_title str
*   review_comment_message str
*   review_creation_date str
*   review_answer_timestamp str

geolocation:
*   geolocation_zip_code_prefix int
*   geolocation_lat str (see note below)
*   geolocation_lng str (see note below)
*   geolocation_city str
*   geolocation_state str

Note: A float or double is a binary value and does not have a well defined number of decimal places.

"Decimal places" have meaning only when you print the number.

To store latitude and longitude values and maintain full accuracy, store either the text string as received from the GPS unit, or store the long integer values maintained internally by TinyGPS. You will lose accuracy if you save the float value. -- from https://forum.arduino.cc/t/storing-latitute-and-longitude-using-a-double/670691/4 

CLEANING REQUIRED (By Table)
*   OK sellers
*   OK customers
*   NOK geolocation (name odd chars)
*   NOK products (missing cat, name, description)
*   OK category_name (ignore english misspellings)
*   OK orders (some missing fields but intended due to delivery status)
*   OK order_items
*   NOK order_reviews (comment title and comment msg blanks n messy)
*   OK order_payments

In [31]:
### PULL CSV from Google Drive
import pandas as pd
# Libraries to read csv file hosted on Google Drive into Google Colab:
!pip install python-tds
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# The shareable links from Google Drive for each CSV file hosted there
sellers_link = '1cG5h5AW0hE7OmDZ0YPtZLVSVDnL7mCkQ' 
category_name_link = '1Y1oiP5U1jBO0Ji5FJyRYKULw1T953obp'
products_link = '1L8a9t3obUYIH0RzCfaCcqil7rkjgHOb-'
order_reviews_link = '1_XXIOg3_rDSGKW61EI7MIZ0UqYYpLzfU'
orders_link = '1PaeE67IdrWocbY5isimVwezk7EQK5pAn'
order_payments_link = '1Tfrf7yof8pYXcK4pFuVRt28rWwW9BW4F'
order_items_link = '1IwOvsLbP6ub9BDsXq-FWbd3d7yl1ZJvI'
geolocation_link = '1yL1iSIwMjskL_kKza6kQH8TuI566lYnu'
customers_link = '10cfto60JXwwIe8lewY2dao3KB9TSOuu8'

# Pull the Google Drive hosted CSVs
print("Loading CSV...")

downloaded = drive.CreateFile({'id':sellers_link}) 
downloaded.GetContentFile('olist_sellers_dataset.csv')  

downloaded = drive.CreateFile({'id':category_name_link}) 
downloaded.GetContentFile('product_category_name_translation.csv')  

downloaded = drive.CreateFile({'id':products_link}) 
downloaded.GetContentFile('olist_products_dataset.csv')  

downloaded = drive.CreateFile({'id':order_reviews_link}) 
downloaded.GetContentFile('olist_order_reviews_dataset.csv')  

downloaded = drive.CreateFile({'id':orders_link}) 
downloaded.GetContentFile('olist_orders_dataset.csv')  

downloaded = drive.CreateFile({'id':order_payments_link}) 
downloaded.GetContentFile('olist_order_payments_dataset.csv')  

downloaded = drive.CreateFile({'id':order_items_link}) 
downloaded.GetContentFile('olist_order_items_dataset.csv')  

downloaded = drive.CreateFile({'id':geolocation_link}) 
downloaded.GetContentFile('olist_geolocation_dataset.csv')  

downloaded = drive.CreateFile({'id':customers_link}) 
downloaded.GetContentFile('olist_customers_dataset.csv')  

print("all CSV downloaded!\n")


Loading CSV...
all CSV downloaded!



In [ ]:
# airflow libs install
!pip install apache-airflow
!pip install apache-airflow[cncf.kubernetes]

In [ ]:
!airflow db init
!airflow version

In [29]:
#!mkdir /content/dags  #remember to add olistDAG.py 
#!airflow tasks run odag tryprint '2022-04-27T00:00:00+00:00'
!airflow dags test odag '2022-04-27T00:00:00+00:00'

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144 UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
[2022-04-28 13:43:24,568] {dagbag.py:500} INFO - Filling up the DagBag from /content/dags
[2022-04-28 13:43:24,631] {tutorial_taskflow_api_etl_virtualenv.py:30} WARNING - The tutorial_taskflow_api_etl_virtualenv example DAG requires virtualenv, please install it.
[2022-04-28 13:43:24,646] {example_python_operator.py:68} WARNING - The virtalenv_python example task requires virtualenv, please install it.
[2022-04-28 13:43:24,809] {base_executor.py:85} INFO - Adding to queue: ['<TaskInstance: odag.tryinit backfill__2022-04-27T00:00:00+00:00 [queued]>']
[2022-04-28 13:43:24,838] {base_executor.py:85} INFO - Adding to queue: ['<TaskInstance: odag.loadpip1 backfill__2022-04-27T00:0